In [1]:
IFD_TOKN = "PPrBwtUvQCWRGBzfQa3TYgcuuFA9EYHJ7r3OKg4VhU9D1zM79oV07rBKxa_Hd-74TOzHKCTZK4S9SAypvo_VyA=="
IFD_ORGN = "artica"
IFD_HOST = "eu-central-1-1.aws.cloud2.influxdata.com"
IFD_ADDR = f"https://{IFD_HOST}"

grafanaToken = "HgPeKvrZrR75TnyDNPbkaMg2wMKRdX7wXPGttlI7-QUmei5dvfxXZD-tnHVpbSxXQNGUF2_U-o4z498qWMQLvw=="

import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

client = influxdb_client.InfluxDBClient(
   url=IFD_ADDR,
   token=IFD_TOKN,
   org=IFD_ORGN
)

write_api = client.write_api(write_options=SYNCHRONOUS)

In [2]:
# 2023-03-18 17:37:40.807603965
# 2023-03-18 17:37:46.291760032

In [3]:
# fields --> standardTag
# measurement --> order-data-1

In [4]:
import pandas as pd
dataDf = pd.read_excel('DATA.xlsx')

record1 = dataDf.to_dict(orient='records')[0]

record1 = {key:record1[key] for key in record1 if key != 'time'}

In [5]:
data = {
    'measurement': 'orderId1',
    'fields'     : record1
}

data

{'measurement': 'orderId1',
 'fields': {'PT2_7_SCALE_PV': 13.59,
  'TOTALISE_FUEL_FLOW': 47432.05,
  'TOTAL_STEAM': 424826.47,
  'TE1_5_SCALE_PV': 103.0}}

In [6]:
# write_api.write(
#     'site-data',
#     IFD_ORGN,
#     [data]
# )

In [21]:
query_api = client.query_api()

import warnings
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

In [30]:
# list measurements/tables in a bucket

query = """
import "influxdata/influxdb/schema"
schema.measurements(bucket: "site-data")
"""

query_api.query_data_frame(query)

,result,table,_value
0,_result,0,orderId1


In [36]:
# list fields in a bucket (so table ain't shit)
query = """
import "influxdata/influxdb/schema"
schema.fieldKeys(bucket: "site-data")
"""

query_api.query_data_frame(query)

,result,table,_value
0,_result,0,PT2_7_SCALE_PV
1,_result,0,TE1_5_SCALE_PV
2,_result,0,TOTALISE_FUEL_FLOW
3,_result,0,TOTAL_STEAM


In [37]:
# list fields in a bucket and in a measurement
query = """
import "influxdata/influxdb/schema"
schema.measurementFieldKeys(bucket: "site-data", measurement: "orderId1")
"""

query_api.query_data_frame(query)

,result,table,_value
0,_result,0,PT2_7_SCALE_PV
1,_result,0,TE1_5_SCALE_PV
2,_result,0,TOTALISE_FUEL_FLOW
3,_result,0,TOTAL_STEAM


In [34]:
# get all values
query = """
from(bucket: "site-data")
  |> range(start: -30d)
"""

query_api.query_data_frame(query)

,result,table,_start,_stop,_time,_value,_field,_measurement
0,_result,0,2023-02-19 09:09:13.458655+00:00,2023-03-21 09:09:13.458655+00:00,2023-03-19 20:13:44.290982+00:00,13.59,PT2_7_SCALE_PV,orderId1
1,_result,1,2023-02-19 09:09:13.458655+00:00,2023-03-21 09:09:13.458655+00:00,2023-03-19 20:13:44.290982+00:00,103.00,TE1_5_SCALE_PV,orderId1
2,_result,2,2023-02-19 09:09:13.458655+00:00,2023-03-21 09:09:13.458655+00:00,2023-03-19 20:13:44.290982+00:00,47432.05,TOTALISE_FUEL_FLOW,orderId1
3,_result,3,2023-02-19 09:09:13.458655+00:00,2023-03-21 09:09:13.458655+00:00,2023-03-19 20:13:44.290982+00:00,424826.47,TOTAL_STEAM,orderId1
